In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Crohns_Disease"
cohort = "GSE66407"

# Input paths
in_trait_dir = "../../input/GEO/Crohns_Disease"
in_cohort_dir = "../../input/GEO/Crohns_Disease/GSE66407"

# Output paths
out_data_file = "../../output/preprocess/Crohns_Disease/GSE66407.csv"
out_gene_data_file = "../../output/preprocess/Crohns_Disease/gene_data/GSE66407.csv"
out_clinical_data_file = "../../output/preprocess/Crohns_Disease/clinical_data/GSE66407.csv"
json_path = "../../output/preprocess/Crohns_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# Analysis of dataset to determine gene expression data availability and clinical feature extraction
import pandas as pd

# 1. Gene Expression Data Availability 
# Based on the background information, this dataset contains gut biopsies with transcriptome analysis
# This indicates gene expression data, not just miRNA or methylation
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Trait (Crohn's Disease) - From key 3 "diagnosis: CD"
trait_row = 3

# Age - From key 2 "age: XX"
age_row = 2

# Gender - Not available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert diagnosis information to binary trait value (0: Control, 1: CD)."""
    if pd.isna(value):
        return None
    
    # Split by colon and get the value
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value == 'CD':
        return 1  # Has Crohn's Disease
    elif value == 'Control':
        return 0  # Control/Healthy
    else:
        return None  # UC or other diagnoses

def convert_age(value):
    """Convert age information to continuous value."""
    if pd.isna(value):
        return None
    
    # Split by colon and get the value
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    """Placeholder function for gender conversion, though gender data is not available."""
    return None

# 3. Save Metadata - initial filtering
# trait_row is not None, so trait data is available
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=(trait_row is not None)
)

# 4. Clinical Feature Extraction (since trait_row is not None)
# Use a safer approach to parse the sample characteristics dictionary from the previous output
sample_char_dict = {0: ['patient: 10', 'patient: 53', 'patient: 22', 'patient: 91', 'patient: 23', 'patient: 96', 'patient: 50', 'patient: 9', 'patient: 25', 'patient: 97', 'patient: 12', 'patient: 52', 'patient: 101', 'patient: 29', 'patient: 51', 'patient: 107', 'patient: 43', 'patient: 11', 'patient: 109', 'patient: 40', 'patient: 113', 'patient: 116', 'patient: 39', 'patient: 120', 'patient: 34', 'patient: 48', 'patient: 59', 'patient: 65', 'patient: 99', 'patient: 28'], 
                    1: ['biopsy: 2', 'biopsy: 3', 'biopsy: 6', 'biopsy: 5', 'biopsy: 1', 'biopsy: 4', 'biopsy: 7', 'biopsy: 8', 'biopsy: G1', 'biopsy: 9', 'biopsy: 1A', 'biopsy: 1B'], 
                    2: ['age: 37', 'age: 18', 'age: 19', 'age: 54', 'age: 70', 'age: 22', 'age: 45', 'age: 62', 'age: 31', 'age: 39', 'age: 67', 'age: 24', 'age: 59', 'age: 20', 'age: 77', 'age: 68', 'age: 41', 'age: 50', 'age: 35', 'age: 36', 'age: 43', 'age: 52', 'age: 21', 'age: 63', 'age: 29', 'age: 25', 'age: 26', 'age: 28', 'age: 53', 'age: 69'], 
                    3: ['diagnosis: Control', 'diagnosis: CD', 'diagnosis: UC', None, 'inflammation: non', 'inflammation: yes'], 
                    4: ['gastroscopy: FALSE', 'gastroscopy: TRUE', None, 'tissue: transversum', 'tissue: sigmoideum'], 
                    5: ['inflammation: non', 'inflammation: yes', None, 'tissue: descendens', 'tissue: sigmoideum', 'tissue: rectum'], 
                    6: ['tissue: ascendens', 'tissue: sigmoideum', 'tissue: ileum', 'tissue: rectum', 'tissue: descendens', 'tissue: transversum', 'tissue: coecum', None, 'tissue: bulbus durodenum', 'tissue: valvula']}

# Create clinical data DataFrame properly
# We need to account for the fact that each list in the dictionary may have different lengths
# Find the maximum length
max_length = max(len(values) for values in sample_char_dict.values())

# Pad shorter lists with NaN
padded_dict = {}
for key, values in sample_char_dict.items():
    padded_values = values + [None] * (max_length - len(values))
    padded_dict[key] = padded_values

# Create DataFrame from padded dictionary
clinical_data = pd.DataFrame(padded_dict)

# Extract clinical features
clinical_features = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Preview clinical features
preview_clinical = preview_df(clinical_features)
print("Preview of extracted clinical features:")
print(preview_clinical)

# Save clinical features to CSV
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical features saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True


### Step 4: Gene Identifier Review

In [ ]:
# Looking at the gene identifiers provided, they appear to be Ensembl gene IDs with an "_at" suffix
# Ensembl IDs typically start with "ENSG" for human genes, followed by a unique number
# These are not standard human gene symbols (like BRCA1, TP53, etc.)
# Therefore, they need to be mapped to standard gene symbols for better interpretability

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Looking at the gene annotation preview, we need to extract gene symbols from 'Description'
# Example: "tetraspanin 6 [Source:HGNC Symbol;Acc:11858]" should yield "TSPAN6"

# Let's check the Description field format
print("\nSample Description fields:")
print(gene_annotation['Description'].head(10).tolist())

# Create a function to extract gene symbols from the Description field
def extract_gene_name(description):
    """Extract gene name from the description field - everything before [Source: part"""
    if pd.isna(description):
        return None
    
    # Extract the gene name - everything before [Source: part
    if '[Source:' in description:
        gene_name = description.split('[Source:')[0].strip()
        return gene_name
    
    return None

# Apply the function to extract gene names
gene_annotation['Gene_Name'] = gene_annotation['Description'].apply(extract_gene_name)

# Print some examples to verify extraction
print("\nSample Gene Name extractions:")
sample_extractions = gene_annotation[['ID', 'Description', 'Gene_Name']].head(10)
print(sample_extractions)

# Use extract_human_gene_symbols to get likely gene symbols from the gene names
gene_annotation['Symbol'] = gene_annotation['Gene_Name'].apply(extract_human_gene_symbols)

# Check which rows have symbols
has_symbols = gene_annotation['Symbol'].apply(lambda x: len(x) > 0 if isinstance(x, list) else False)
print(f"\nRows with extracted symbols: {has_symbols.sum()} out of {len(gene_annotation)}")

# Create an exploded dataframe for mapping
mapping_df = gene_annotation[['ID', 'Symbol']].copy()
# Convert empty lists to None to make dropna work correctly
mapping_df.loc[mapping_df['Symbol'].apply(lambda x: isinstance(x, list) and len(x) == 0), 'Symbol'] = None
# Explode the Symbol column to handle cases where multiple symbols were extracted
mapping_df = mapping_df.explode('Symbol')
mapping_df = mapping_df.dropna(subset=['Symbol'])
mapping_df = mapping_df.rename(columns={'Symbol': 'Gene'})

# Print the shape of the mapping dataframe
print(f"\nMapping dataframe shape: {mapping_df.shape}")
print("\nSample mappings:")
print(mapping_df.head(10))

# Apply the mapping to convert probe-level data to gene expression data
gene_data_mapped = apply_gene_mapping(gene_data, mapping_df)

# Print the shape of the resulting gene expression data
print(f"\nAfter mapping, gene data dimensions: {gene_data_mapped.shape[0]} genes × {gene_data_mapped.shape[1]} samples")

# Preview the first few gene symbols
if gene_data_mapped.shape[0] > 0:
    print("\nFirst few gene symbols after mapping:")
    print(gene_data_mapped.index[:10])
else:
    print("\nWarning: No genes were mapped. Check mapping process.")

# Save the gene expression data
gene_data_mapped.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")

# Update the gene_data variable for subsequent steps
gene_data = gene_data_mapped


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Re-extract clinical data from the SOFT file to ensure we have properly structured data
print("Re-extracting clinical data from the SOFT file...")
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Get raw clinical data
background_info, clinical_df = get_background_and_clinical_data(matrix_file)

# Transpose the clinical data to have samples as columns
clinical_df_t = clinical_df.T
# First row contains the geo accessions, use it as column names
clinical_df_t.columns = clinical_df_t.iloc[0]
# Remove the first row that has been used as column names
clinical_df_t = clinical_df_t.iloc[1:]

# Now extract relevant clinical features (for Crohn's Disease and age)
trait_values = []
if trait_row is not None:
    for col in clinical_df_t.columns:
        trait_val = convert_trait(clinical_df_t.iloc[trait_row-1, col])  # Adjust index for 0-based
        trait_values.append(trait_val)

age_values = []
if age_row is not None:
    for col in clinical_df_t.columns:
        age_val = convert_age(clinical_df_t.iloc[age_row-1, col])  # Adjust index for 0-based
        age_values.append(age_val)

# Create proper clinical features dataframe with samples as rows
sample_ids = clinical_df_t.columns.tolist()
clinical_features = pd.DataFrame()

if trait_values:
    clinical_features[trait] = trait_values
if age_values:
    clinical_features['Age'] = age_values

# Set index to sample IDs
clinical_features.index = sample_ids
print(f"Re-extracted clinical features shape: {clinical_features.shape}")
print("Clinical features preview:")
print(clinical_features.head())

# Save the improved clinical features
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Improved clinical features saved to {out_clinical_data_file}")

# Check if clinical features were successfully extracted with non-null values
if clinical_features.empty or clinical_features[trait].isnull().all():
    print("Failed to extract valid clinical features with trait values. Dataset cannot be processed further.")
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=True,
        df=pd.DataFrame(),
        note="Valid clinical features with trait values could not be extracted."
    )
else:
    # 2. Link clinical and genetic data
    print("Linking clinical and genetic data...")
    
    # Transpose gene_data to have samples as rows and genes as columns
    gene_data_t = gene_data.T
    
    # Keep only common samples between clinical and gene data
    common_samples = list(set(clinical_features.index) & set(gene_data_t.index))
    print(f"Common samples between clinical and gene data: {len(common_samples)}")
    
    if len(common_samples) == 0:
        print("No common samples between clinical and gene data. Dataset cannot be processed further.")
        is_usable = validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=True,
            is_trait_available=True,
            is_biased=True,
            df=pd.DataFrame(),
            note="No common samples between clinical and gene data."
        )
    else:
        # Filter both datasets to only include common samples
        clinical_common = clinical_features.loc[common_samples]
        gene_data_common = gene_data_t.loc[common_samples]
        
        # Merge the datasets
        linked_data = pd.concat([clinical_common, gene_data_common], axis=1)
        print(f"Linked data shape: {linked_data.shape}")
        
        # 3. Handle missing values systematically
        linked_data = handle_missing_values(linked_data, trait_col=trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
        
        # Check if there are still samples after missing value handling
        if linked_data.shape[0] == 0:
            print("Error: No samples remain after handling missing values.")
            is_usable = validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=True,
                is_trait_available=True,
                is_biased=True,
                df=pd.DataFrame(),
                note="All samples were removed during missing value handling."
            )
        else:
            # 4. Check if the dataset is biased
            print("\nChecking for bias in feature variables:")
            is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
            
            # 5. Conduct final quality validation
            is_usable = validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=True,
                is_trait_available=True,
                is_biased=is_biased,
                df=linked_data,
                note="Dataset contains gene expression data for Crohn's Disease."
            )
            
            # 6. Save linked data if usable
            if is_usable:
                os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                linked_data.to_csv(out_data_file)
                print(f"Linked data saved to {out_data_file}")
            else:
                print("Dataset deemed not usable for trait association studies, linked data not saved.")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Skip gene symbol normalization and use the accession numbers directly
print("Processing gene expression data...")
# Don't normalize - these are GenBank accessions, not gene symbols
gene_data_normalized = gene_data  # Use the original gene data with accession numbers

# Save the gene data (without normalization)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")
print(f"Gene data shape: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# 2. Extract clinical features from scratch
print("Extracting clinical features from original clinical data...")
clinical_features = geo_select_clinical_features(
    clinical_data, 
    trait, 
    trait_row,
    convert_trait,
    age_row,
    convert_age,
    gender_row,
    convert_gender
)

# Save the extracted clinical features
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical features saved to {out_clinical_data_file}")

print("Clinical features preview:")
print(preview_df(clinical_features))

# Check if clinical features were successfully extracted
if clinical_features.empty:
    print("Failed to extract clinical features. Dataset cannot be processed further.")
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=True,
        df=pd.DataFrame(),
        note="Clinical features could not be extracted from the dataset."
    )
    print("Dataset deemed not usable due to lack of clinical features.")
else:
    # 2. Link clinical and genetic data
    print("Linking clinical and genetic data...")
    linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
    print(f"Linked data shape: {linked_data.shape}")

    # Check if the linked data has gene features
    if linked_data.shape[1] <= 1:
        print("Error: Linked data has no gene features. Dataset cannot be processed further.")
        is_usable = validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=False,
            is_trait_available=True,
            is_biased=True,
            df=linked_data,
            note="Failed to link gene expression data with clinical features."
        )
    else:
        # 3. Handle missing values systematically
        linked_data = handle_missing_values(linked_data, trait_col=trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
        
        # Check if there are still samples after missing value handling
        if linked_data.shape[0] == 0:
            print("Error: No samples remain after handling missing values.")
            is_usable = validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=True,
                is_trait_available=True,
                is_biased=True,
                df=pd.DataFrame(),
                note="All samples were removed during missing value handling."
            )
        else:
            # 4. Check if the dataset is biased
            print("\nChecking for bias in feature variables:")
            is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

            # 5. Conduct final quality validation
            is_usable = validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=True,
                is_trait_available=True,
                is_biased=is_biased,
                df=linked_data,
                note="Dataset contains gene expression data for Crohn's Disease patients and healthy controls."
            )

            # 6. Save linked data if usable
            if is_usable:
                os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                linked_data.to_csv(out_data_file)
                print(f"Linked data saved to {out_data_file}")
            else:
                print("Dataset deemed not usable for trait association studies, linked data not saved.")